<a href="https://colab.research.google.com/github/danidmvz/SPARSEwithRL/blob/main/SPARSEsine1D/SPARSEsine1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SPARSE with stochastic forcing: what is the optimal forcing function?** 

In this code I present a one-dimensional version of the SPARSE method with stochastic forcing (S-SPARSE) to describe the agent that is trying to minimize some Quantity of Interest (QoI) as it might be the particle spread (standard deviation of the particle positions) $\sigma_{x_p}$.

The agent: macro-particle, learns the stochastic forcing function $f(\boldsymbol \alpha,\boldsymbol a)=\sum_i^{N}\alpha_i \Psi_i(\boldsymbol a)$ and in particular its coefficients stuck in the vector $\boldsymbol \alpha$. The basis functions $\Psi_i$ are chosen according to the Chebyshev modes of the first kind of a correlation. In this case I use the Schiller and Naumann correlation $f(Re_p)=1+0.15Re_p^{0.687}$.

The agent learns to control the second moments of the stochastic coefficients $\overline{\alpha_i^\prime \alpha_j^\prime}$ to minimize the cost function that is defined such that the particle spread is minimum. The agent is trained using maximum a posteriori policy optimization ([mpo](https://arxiv.org/abs/1806.06920)). The deep reinforcment learning framework [acme](https://github.com/deepmind/acme), developed by DeepMind, is used to train the agent.

The notebook describes in order: 
- The environment components and  the implementation architecture required by [dm_control](https://github.com/deepmind/dm_control).
- The agent set-up and training loop. 
- The evaluation of the trained agent when applied to the environment. 

# Install acme


In [3]:
# Install acme
!pip install virtualenv
!pip install --upgrade pip setuptools wheel
!virtualenv .acme
!source .acme/bin/activate

# A fixed release version of dm-acme and dm-control is enforced for long term maintenance
!pip install dm-acme[tf]==0.3.0 
!pip install dm-control==0.0.364896371

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 40.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.0 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confli

In [ ]:
# Import
from dm_control.rl.control import Environment
import numpy as np
import matplotlib.pyplot as plt

from acme import agents, specs
from acme.environment_loop import EnvironmentLoop
from acme.wrappers.single_precision import SinglePrecisionWrapper
from acme.wrappers.canonical_spec import CanonicalSpecWrapper
from acme.agents.tf import mpo
from acme.tf import networks
from acme.tf import utils as tf2_utils
import sonnet as snt
from acme.utils import paths
import tensorflow as tf 
from acme.utils.loggers import tf_summary
from __future__ import annotations
import numpy as np
from dm_control.rl import control


#  **Water tank environment**

## Physics model
The ordinary differential equation governing the evolution of the water level $h$ in the tank, provided the appropriate physical scaling, can be written as,
\begin{align}
\frac{d h}{d t} = w_{out} + w_{in}.
\end{align}

The following constitutive equation is assumed for the water outflow in the nozzle,
\begin{align}
w_{out} = - \alpha \sqrt{h}.
\end{align}

The equation is discretized in time with Euler explicit scheme,
\begin{align}
h^{t+1} = dt_{sim}*(w_{out}^{t} + w_{in}^{t}) + h^{t}  
\end{align}


## Environment in dm_control framework

To make use of [acme](https://github.com/deepmind/acme) architecture for continuous control purposes it's convenient to implement the environment following the [dm_control](https://github.com/deepmind/dm_control) architecture. This allows to benefit from several tools and routines to simplify the set-up of the training.

The [dm_control](https://github.com/deepmind/dm_control) framework requires to define the environment as a combination of a **physics** simular and one or multiple **tasks**. 

---


### physics
The main component of the **physics** simulator, given the actions, is to step in time the ode. This is done in the `step` method.

Numerical checks for the solution as well as sanity checks to avoid non physical state of the system are included in the `check_divergence` method.  

In [ ]:
class Physics(control.Physics):
    """Water tank environment built on the dm_control.Environment class."""
    def __init__(
        self,
        dt: float,
        init_state: float,
        init_action: float,
        min_Rep: float,
        max_Rep: float,
        Re_inf: float,
        dp: float,
        St: float,
        ):
        """Initializes water tank

        Attributes:
            alpha: nozzle outflow coefficient
            dt:  [s] Discretization time interval for sim
            hmax: [m] max water height in tank
            init_state: [m] initial water height        
        """
        self._dt = dt
        self._init_state = init_state
        self._state = np.asarray(self._init_state)
        self._time = 0.
        self._init_action = init_action #  np.asarray([0. ,0.])
        self._action = np.asarray(self._init_action)
        self._min_Rep = min_Rep
        self._max_Rep = max_Rep
        self._Re_inf = Re_inf
        self._dp = dp
        self._St = St

    def reset(self):
        """Resets environment physics"""
        self._state = self._init_state
        self._time = 0.
        self._action = self._init_action #  np.zeros((2,1)) # np.asarray([0., 0.])


    def after_reset(self):
        pass


    def step(self, n_sub_steps = 1):
        """Updates the environment according to the action"""

        # Euler explicit time step
        #print(self._dt*self._RHS())
        self._state =  self._state + self._dt*self._RHS()
        #print(self._state)

        # Update sim time
        self._time += self._dt

        # Keep h min at 0
        # if self._state[0] <= 0.: self._state[0] = 0.


    def _RHS(self):
        """ Returns Physical RHS for ODE d state / dt = RHS(state, action) """
        # State variables
        mean_xp = self._state[0]
        mean_up = self._state[1]
        prime_xpxp = self._state[2]
        prime_xpup = self._state[3]
        prime_upup = self._state[4]
        prime_alpha1xp = self._state[5]
        prime_alpha2xp = self._state[6]
        prime_alpha3xp = self._state[7]
        prime_alpha4xp = self._state[8]
        prime_alpha5xp = self._state[9]
        prime_alpha1up = self._state[10]
        prime_alpha2up = self._state[11]
        prime_alpha3up = self._state[12]
        prime_alpha4up = self._state[13]
        prime_alpha5up = self._state[14]

        # Action variables 
        prime_alpha1alpha1 = self._action[0]
        prime_alpha1alpha2 = self._action[1]
        prime_alpha1alpha3 = self._action[2]
        prime_alpha1alpha4 = self._action[3]
        prime_alpha1alpha5 = self._action[4]
        prime_alpha2alpha2 = self._action[5]
        prime_alpha2alpha3 = self._action[6]
        prime_alpha2alpha4 = self._action[7]
        prime_alpha2alpha5 = self._action[8]
        prime_alpha3alpha3 = self._action[9]
        prime_alpha3alpha4 = self._action[10]
        prime_alpha3alpha5 = self._action[11]
        prime_alpha4alpha4 = self._action[12]
        prime_alpha4alpha5 = self._action[13]
        prime_alpha5alpha5 = self._action[14]

        # Flow
        u_at_mean_xp, dudx_at_mean_xp, d2udx_at_mean_xp = self._Flow()
        mean_u = u_at_mean_xp + 0.5*( prime_xpxp*d2udx_at_mean_xp )
        #print('flow',u_at_mean_xp)

        # Terms xpu
        prime_xpu = prime_xpxp*dudx_at_mean_xp

        # Terms upu
        prime_upu = prime_xpup*dudx_at_mean_xp

        # Terms uu
        prime_uu = prime_xpu*dudx_at_mean_xp

        # Terms u with coefficients
        prime_alpha1u = prime_alpha1xp*dudx_at_mean_xp;
        prime_alpha2u = prime_alpha2xp*dudx_at_mean_xp;
        prime_alpha3u = prime_alpha3xp*dudx_at_mean_xp;
        prime_alpha4u = prime_alpha4xp*dudx_at_mean_xp;
        prime_alpha5u = prime_alpha5xp*dudx_at_mean_xp;

        # Terms with relative velocity
        mean_ax = mean_u - mean_up
        prime_alpha1ax = prime_alpha1u - prime_alpha1up;
        prime_alpha2ax = prime_alpha2u - prime_alpha2up;
        prime_alpha3ax = prime_alpha3u - prime_alpha3up;
        prime_alpha4ax = prime_alpha4u - prime_alpha4up;
        prime_alpha5ax = prime_alpha5u - prime_alpha5up;

        # Forcing
        f_at_means = self._Forcing(mean_ax)
        mean_f = 1 # f_at_means + 0.5*( 1 ) # This is wrong

        # Forcing with particle position
        prime_xpf = prime_alpha1xp*0 # This is wrong
          
        # Forcing with particle velocity 
        prime_upf = prime_alpha1up*0 # This is wrong

        # Forcing with flow
        prime_uf = prime_xpf*dudx_at_mean_xp

        # Forcing with coefficients
        prime_alpha1f = prime_alpha2alpha2 # This is wrong
        prime_alpha2f = prime_alpha2alpha2 # This is wrong
        prime_alpha3f = prime_alpha3alpha3 # This is wrong
        prime_alpha4f = prime_alpha4alpha4 # This is wrong
        prime_alpha5f = prime_alpha5alpha5 # This is wrong


        # Printing
        #print('St',type(self._St))
        #print('mean_xp', type(mean_xp))
        #print('mean_up', type(mean_up))
        #print('prime_xpxp', type(prime_xpxp))
        #print('prime_xpup', type(prime_xpup))
        #print('prime_upup', type(prime_upup))
        #print('prime_alpha1xp', type(prime_alpha1xp))
        #print('prime_alpha2xp', type(prime_alpha2xp))
        #print('prime_alpha3xp', type(prime_alpha3xp))
        #print('prime_alpha4xp', type(prime_alpha4xp))
        #print('prime_alpha5xp', type(prime_alpha5xp))
        #print('prime_alpha1up', type(prime_alpha1up))
        #print('prime_alpha2up', type(prime_alpha2up))
        #print('prime_alpha3up', type(prime_alpha3up))
        #print('prime_alpha4up', type(prime_alpha4up))
        #print('prime_alpha5up', type(prime_alpha5up))

        # Equations
        ddt_mean_xp =  1.*mean_up 
        #print('mean_up')
        #print( (1./self._St)*( mean_f*(mean_u-mean_up) + prime_uf - prime_upf ) )
        ddt_mean_up =  (1./self._St)*( mean_f*(mean_u-mean_up) + prime_uf - prime_upf ) 
        ddt_prime_xpxp =  2.*prime_xpup 
        ddt_prime_xpup =  prime_upup + ( mean_f*(prime_xpu-prime_xpup) + prime_xpf*(mean_u-mean_up) )/self._St 
        ddt_prime_upup = (  mean_f*(2.*prime_upu-2.*prime_upup) + 2.*prime_upf*(mean_u-mean_up) )*(1./self._St) 
        ddt_prime_alpha1xp =  prime_alpha1up 
        ddt_prime_alpha2xp =  prime_alpha2up 
        ddt_prime_alpha3xp =  prime_alpha3up 
        ddt_prime_alpha4xp =  prime_alpha4up 
        ddt_prime_alpha5xp =  prime_alpha5up 
        ddt_prime_alpha1up =  ( mean_f*(prime_alpha1u-prime_alpha1up) + prime_alpha1f*(mean_u-mean_up) )/self._St 
        ddt_prime_alpha2up =  ( mean_f*(prime_alpha2u-prime_alpha2up) + prime_alpha2f*(mean_u-mean_up) )/self._St 
        ddt_prime_alpha3up =  ( mean_f*(prime_alpha3u-prime_alpha3up) + prime_alpha3f*(mean_u-mean_up) )/self._St 
        ddt_prime_alpha4up =  ( mean_f*(prime_alpha4u-prime_alpha4up) + prime_alpha4f*(mean_u-mean_up) )/self._St 
        ddt_prime_alpha5up =  ( mean_f*(prime_alpha5u-prime_alpha5up) + prime_alpha5f*(mean_u-mean_up) )/self._St 
        
        #print('ddt_mean_xp', type(ddt_mean_xp))
        #print('ddt_mean_up', type(ddt_mean_up))
        #print('ddt_prime_xpxp', type(ddt_prime_xpxp))
        #print('ddt_prime_xpup', type(ddt_prime_xpup))
        #print('ddt_prime_upup', type(ddt_prime_upup))
        #print('ddt_prime_alpha1xp', type(ddt_prime_alpha1xp))
        #print('ddt_prime_alpha2xp', type(ddt_prime_alpha2xp))
        #print('ddt_prime_alpha3xp', type(ddt_prime_alpha3xp))
        #print('ddt_prime_alpha4xp', type(ddt_prime_alpha4xp))
        #print('ddt_prime_alpha5xp', type(ddt_prime_alpha5xp))
        #print('ddt_prime_alpha1up', type(ddt_prime_alpha1up))
        #print('ddt_prime_alpha2up', type(ddt_prime_alpha2up))
        #print('ddt_prime_alpha3up', type(ddt_prime_alpha3up))
        #print('ddt_prime_alpha4up', type(ddt_prime_alpha4up))
        #print('ddt_prime_alpha5up', type(ddt_prime_alpha5up))

        rhs = np.asarray([ddt_mean_xp, ddt_mean_up, ddt_prime_xpxp, ddt_prime_xpup, ddt_prime_upup, 
               ddt_prime_alpha1xp, ddt_prime_alpha2xp, ddt_prime_alpha3xp, ddt_prime_alpha4xp, ddt_prime_alpha5xp,
               ddt_prime_alpha1up, ddt_prime_alpha2up, ddt_prime_alpha3up, ddt_prime_alpha4up, ddt_prime_alpha5up])
        #print(rhs)
        #print(type(rhs))
        return rhs
        # return np.append(ddt_mean_xp, ddt_mean_xp, ddt_prime_xpxp, ddt_prime_xpup, ddt_prime_upup, 
                         # ddt_prime_alphaxp1, ddt_prime_alphaxp2, ddt_prime_alphaxp3, ddt_prime_alphaxp4, ddt_prime_alphaxp5,
                         # ddt_prime_alphaup1, ddt_prime_alphaup2, ddt_prime_alphaup3, ddt_prime_alphaup4, ddt_prime_alphaup5)
        

    def _Flow(self):
      u0 = 1
      A = 0.5
      k = 2
      w = 0
      phi = 0 
      t = self._time
      x = self._state[0]
      u = u0 + A*np.sin(phi - t*w + k*x)
      dudx = A*k*np.cos(phi - t*w + k*x)
      d2udx = -A*k**2*np.sin(phi -t*w + k*x)
      return float(u), float(dudx), float(d2udx)


    def _Forcing(self, mean_ax):
        # Action variables 
        prime_alpha1alpha1 = self._action[0]
        prime_alpha1alpha2 = self._action[1]
        prime_alpha1alpha3 = self._action[2]
        prime_alpha1alpha4 = self._action[3]
        prime_alpha1alpha5 = self._action[4]
        prime_alpha2alpha2 = self._action[5]
        prime_alpha2alpha3 = self._action[6]
        prime_alpha2alpha4 = self._action[7]
        prime_alpha2alpha5 = self._action[8]
        prime_alpha3alpha3 = self._action[9]
        prime_alpha3alpha4 = self._action[10]
        prime_alpha3alpha5 = self._action[11]
        prime_alpha4alpha4 = self._action[12]
        prime_alpha4alpha5 = self._action[13]
        prime_alpha5alpha5 = self._action[14]
        #print(self._Re_inf)
        #print(mean_ax)
        #print(self._dp)
        
        Rep = self._Re_inf*np.asarray(np.absolute(mean_ax), dtype=float)*self._dp
        #a = self._min_Rep
        #b = self._max_Rep
        #n = 5
        #N = n+1
        #i = np.linspace(1, N, num=N)
        #u_nodes = np.cos((2*i-1)*np.pi/(2*N))
        #x_nodes = 0.5*(b - a)*u_nodes + 0.5*(b+a);
        #y_nodes = 1 + 0.15*x_nodes**0.687;

        #A = np.zeros((n,))
        #A[0] = 1
        #A[1:] = 2

        #Tn1 = 1 + 0*u_nodes
        #Tn2 = u_nodes
        #Tn3 = 2*u_nodes**2 - 1
        #Tn4 = 4*u_nodes**3 - 3*u_nodes
        #Tn5 = 8*u_nodes**4 - 8*u_nodes**2 + 1

        #c1 = A[1]*np.average(Tn1*y_nodes)
        #c2 = A[2]*np.average(Tn2*y_nodes)
        #c3 = A[3]*np.average(Tn3*y_nodes)
        #c4 = A[4]*np.average(Tn4*y_nodes)
        #c5 = A[5]*np.average(Tn5*y_nodes)

        # For now this works for one macro-particle, I think we need a loop to 
        # extend it to several

        return 1. # + 0.15*Rep**0.687 + prime_alpha1alpha1


    def time(self):
        """Returns total elapsed simulation time"""
        return self._time


    def timestep(self):
        """Returns dt simulation step"""
        return self._dt


    def check_divergence(self):
        """ Checks physical terminations:
         - water reached maximum level
         - physical states not finite
         """
        #if  self._state[0] >= self_hmax:
        #    raise control.PhysicsError(
        #        f'h > max value = {self._hmax} [m]'
        #    )
        #if not all(np.isfinite(self._state)):
        #    raise control.PhysicsError('System state not finite')
        pass

    def set_control(self, action):
        """Sets control actions""" 
        self._action = action 


    def get_state(self):
        """Returns physical states"""
        return self._state

### task
Each **task** has several purposes:

-  Initialize the physics.

For example, initializing the initial water level. This could differ from task to task.

-  Define the reward function, hence the control target

In this tutorial we will consider a Step target, which aims to keep the water level constant during a first time interval and then step the level to a different constant target. The reward function is defined as a normal distribution, with the mean equal to the target water level at a given time and the standard deviation $\sigma$ set to $0.05m$. A well trained agent should be able to control the water level with a precision $\sim \sigma$. 

-  Provide the observations to be sent to the actor given the state of the system and the control target for the task.

Each task could potentially observe a different subset of the system state. In this tutorial the agent will observe directly the water level, hence the full physical state. In order to let the agent learn how to deal with a time varying target, together with the physical state, we let the system observe the water level desired target.

- Define physical limits for the actions.   


In [ ]:
class Step(control.Task):
    """ Step task:
    Keep constant value and step to different constant value at t_step
    """

    def __init__(self,
                 precision: float,
                 targets,
                 ):
        """Initialize Step task
        
        Parameters:
            precision: [m] desired precision on h target   
        
        """
        self._precision = precision
        self._targets = targets


    def initialize_episode(self, physics):
        """ Reset physics for the task """
        physics.reset()

    def get_reference(self, physics):
        """Returns target reference"""
        # self._targets[0] = 0.05 # Target prime_xpxp  
        return self._targets

    def get_observation(self, physics):
        """Returns specific observation for the task"""
        return np.concatenate((self.get_reference(physics), physics.get_state()))

    def get_reward(self, physics):
        """Returns the reward given the physical state """
        sigma = self._precision
        mean = self.get_reference(physics)
        rewardFunc = np.exp(-((physics.get_state()[3] - mean[0])**2.)/(2*sigma**2.))
        return rewardFunc 

    def before_step(self, action, physics):
        physics.set_control(action)
     
    def observation_spec(self, physics):
        """Returns the observation specifications"""
        return specs.Array(
            shape = (16,), # 15 states plus the reference, in this case the single target
            dtype = np.float32,
            name = 'observation')

    def action_spec(self, physics):
        """Returns the action specifications"""
        return specs.BoundedArray(
            shape = (15,), # there are 15 actions, all the primes_alpha1alpha2
            dtype = np.float32,
            minimum = -10.,
            maximum = 10.,
            name = 'action')


## Simulate the environment with null actions
We can now instantiate the environment and simulate it with null actions to get some intuition of the different components.

Instance of `physics`, `task` and `environment`.

In [ ]:
physics = Physics(
    dt = 0.05,  
    init_state = np.zeros((15,)).tolist(),  
    init_action = np.ones((15,)).tolist(),
    min_Rep = [0.],
    max_Rep = [50.],
    Re_inf = [10000.],
    dp = [np.sqrt(18.*0.5/(10000.*250.))], # np.sqrt(18*St/(Re_inf*rhop))
    St = 0.5,
    )

task = Step(
    precision = 0.05, 
    targets = [0.1],
    #targets = [0.],
)

environment = Environment(
    physics,
    task,
    time_limit = 10, 
) 

Simulate with null action

In [ ]:
TimeStep.observation.tolist()

In [ ]:
s[10]

In [ ]:
# Reset the environment
TimeStep = environment.reset()
    
# Define constant 0 actions
actions = np.zeros(environment.action_spec().shape, np.float32)

# Simulate environment and store state,observation,reward,time
s, o, r, t = [],[],[],[]
mean_xp, mean_up, prime_xpxp, prime_xpup, prime_upup, prime_alpha1xp, prime_alpha2xp, prime_alpha3xp, prime_alpha4xp, prime_alpha5xp, prime_alpha1up, prime_alpha2up, prime_alpha3up, prime_alpha4up, prime_alpha5up = [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

while not TimeStep.last():
  s.append(environment._physics._state)
  o.append(TimeStep.observation.tolist())
  r.append(TimeStep.reward)
  t.append(environment.physics.time())
  TimeStep = environment.step(actions)
  mean_xp.append((environment._physics._state[0]))
  mean_up.append((environment._physics._state[1]))
  prime_xpxp.append((environment._physics._state[2]))
  prime_xpup.append((environment._physics._state[3]))
  prime_upup.append((environment._physics._state[4]))
  prime_alpha1xp.append((environment._physics._state[5]))
  prime_alpha2xp.append((environment._physics._state[6]))
  prime_alpha3xp.append((environment._physics._state[7]))
  prime_alpha4xp.append((environment._physics._state[8]))
  prime_alpha5xp.append((environment._physics._state[9]))
  prime_alpha1up.append((environment._physics._state[10]))
  prime_alpha2up.append((environment._physics._state[11]))
  prime_alpha3up.append((environment._physics._state[12]))
  prime_alpha4up.append((environment._physics._state[13]))
  prime_alpha5up.append((environment._physics._state[14]))


As expected the water level drops from the initial level $h = 1$ to $0$.

In [ ]:
environment._physics._state

In [ ]:
s[0]

In [ ]:
# Plot system state evolution
plt.subplot(1, 2, 1)
plt.plot(t,mean_xp)
plt.plot(t,mean_up)
plt.ylabel('mean x_p, mean u_p')
plt.xlabel('t')

plt.subplot(1, 2, 2)
plt.plot(t,prime_xpxp)
plt.plot(t,prime_xpup)
plt.plot(t,prime_upup)
plt.ylabel('xpxp, xpup, upup')
plt.xlabel('t')

We can check the observations in time that the task `Step` will provide to the agent. As you can see in the figure, the `target` level is provided to the observations together with the `state` to allow the agent learning to follow the target in time.

In [ ]:
# Plot observation
plt.plot(t,o)
plt.ylabel('observations')
plt.xlabel('time [s]')
plt.legend(['target','state'])

We can also check the time trace of the rewards provided by the task. Given the normal definition centered at the target, the reward drops while the water level gets further from the target. 

In [ ]:
# Plot reward
plt.plot(t,r)
plt.ylabel('reward')
plt.xlabel('time [s]')

# **MPO Agent**

In the following section we will show how to train an MPO agent for the environment and task. 
The section is taken mainly from the official [acme](https://github.com/deepmind/acme) tutorial in the following colab <a href="https://colab.research.google.com/github/deepmind/acme/blob/master/examples/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>. We invite to check the official tutorial for a more detailed explanation. 

In [ ]:
# Set random seed for example reproducibility
tf.random.set_seed(1500)

First of all we make use of wrappers to wrap the environment in order to bound the allowed actions within specifications defined in task. We also cast the I/O of the environment into single precision. 

In [ ]:
# Clip actions by bounds
environment = CanonicalSpecWrapper(environment= environment,clip= True) 

# Wrap to single precision
environment = SinglePrecisionWrapper(environment) 

# Extract environment specifications
environment_spec = specs.make_environment_spec(environment)

## Set-up

The next step is to set-up the NN used by the agent. MPO is a somewhat complicated algorithm, so we'll leave a full explanation of this method to the accompanying  [paper](https://arxiv.org/abs/1806.06920). 

However, we give here below some hints to understand the meaning of the following code. The `mpo` is an actor-critic algorithm. 
- The `actor` provides the actions given the state (of the agent). In this simple example the state of the agent consists directly on the observations, i.e. the state of the physical system (water level) and the target level. The actor contains the policy, which ultimately represents the control low to decide the inflow given the actor state (water level).  
- The `critic` learns the state-action value function, which is related to the expected sum of future rewards, given a certain state and a taken action. This function provides intuitively the information on the value of taking a certain action being on a certain state. The critic is used during the learning process to update the policy. 

The `actor` and `critic` functions are approximated with NNs as [sonnet](https://github.com/deepmind/sonnet) MLP modules.

MPO uses a distributional `actor`, as it can be seen from the `MultivariateNormalDiagHead`, which means that the policy obtained is not deterministic. The `policy_network` will return the mean and standard deviation of Normal distribution and the `actor` will sample from this distribution in order to get the actions to be applied to the environment. 

The [acme](https://github.com/deepmind/acme) architecture provides the possibility to specify a neural network for the observations. This is useful for example when dealing with observations coming from images to distil a simple agent state from pixel like information to be given to the critic. In this tuorial, the single physical state $h$ is directly observed and the observation network is simply an identity. 

The `multiplex` combines the actions and observations to be given as inputs to the critic network. 

Overall in general, for the mpo algorithm, one needs to specify 3 NNs for the `actor(policy)_net`, the `critic_net` and  the `observation_net`. 



In [ ]:
# Get total number of action dimensions from action spec.
num_dimensions = np.prod(environment_spec.actions.shape, dtype=int)
num_dimensions

In [ ]:
# Create the shared observation network; here simply a state-less operation.
observation_network = tf2_utils.batch_concat
observation_network

In [ ]:
# Specify default dimension for MLP
policy_layer_sizes = [16,16]
critic_layer_sizes = [16,16]

# Create the policy network.
policy_network = snt.Sequential([
  networks.LayerNormMLP(policy_layer_sizes),
  networks.MultivariateNormalDiagHead(num_dimensions),
])
policy_network

In [ ]:
critic_layer_sizes = list(critic_layer_sizes) + [1] # Hack to conform to mpo implementation
# Create the critic network
critic_network = networks.CriticMultiplexer(critic_network=networks.LayerNormMLP(critic_layer_sizes))
critic_layer_sizes
critic_network

In [ ]:
# Pack agent networks
agent_networks = {
      'policy': policy_network,
      'critic': critic_network,
      'observation': observation_network,
  }
agent_networks

In [ ]:
# Get total number of action dimensions from action spec.
num_dimensions = np.prod(environment_spec.actions.shape, dtype=int)

# Create the shared observation network; here simply a state-less operation.
observation_network = tf2_utils.batch_concat

# Specify default dimension for MLP
policy_layer_sizes = [16,16]
critic_layer_sizes = [16,16]

# Create the policy network.
policy_network = snt.Sequential([
  networks.LayerNormMLP(policy_layer_sizes),
  networks.MultivariateNormalDiagHead(num_dimensions),
])

critic_layer_sizes = list(critic_layer_sizes) + [1] # Hack to conform to mpo implementation
# Create the critic network
critic_network = networks.CriticMultiplexer(critic_network=networks.LayerNormMLP(critic_layer_sizes))

# Pack agent networks
agent_networks = {
      'policy': policy_network,
      'critic': critic_network,
      'observation': observation_network,
  }


Having specified the network architectures, we can finally define the agent, which combines the environment, the actor and the critic. Internally, the MPO agent contains the learner to update the policy.

In [ ]:
environment_spec

In [ ]:
agent = mpo.MPO(
 environment_spec = environment_spec,
 policy_network = agent_networks['policy'],
 critic_network = agent_networks['critic'],
 observation_network = agent_networks['observation'], 
 batch_size = 40,
 target_policy_update_period = 5,
 target_critic_update_period = 5,
 min_replay_size = 10,
 checkpoint = False,
)
    

We define a tensorboard logger to store logs during training and inspect the learning curve afterwards.

In [ ]:
outpath = '/content' # Destination of tensorboard log file
logger = tf_summary.TFSummaryLogger(logdir = outpath)

## Training
Finally we can train the agent. (With 200 episodes it will take ~2 min).

In [ ]:
num_episodes = 200 

# Run the environment loop.
loop = EnvironmentLoop(environment, agent, logger = logger)

# 350 is a good trainin
loop.run(num_episodes=200)

# Visualize training logs with tensorboard

It is convenient to visualize the training results in tensorboard. 
The main plot to observe is the episode return increase (~sum of the reward during an episode) over episodes, which tells how fast the agent is learning. 

Given that the simulation was set to last $2.5s$ with a $dt_{sim}$ of $0.05s$, we expect each episode to last $50$ time steps unless a physical limit defined in `physics` is hit, such as if the water exceed the maximum height.

The maximum reward per time step is $1$, hence the maximum return per episode, given by the sum of the discounted rewards, is $\sim50$. (discount factor is set to 0.99).

We can see that the trained agent achieves an episode return of $\sim 45$.

Another interesting plots is the `StepsPerSecond` which tells how fast is the simulation of a single physical time step. Improving the speed of the environment allows to accelerate the collection of the experience for the learning process. 


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content

# Trained agent evaluation

Now that we trained an agent and inspected its learning properties, we can use the policy obtained and evaluate how it performs in controlling the water level in the `tank`. 

First we simulate the environment with the trained policy.

In [ ]:
# Evaluate 
TimeStep = environment.reset()
# Run episode and store system state, observation, action, reward and time
s, o, a, r, t = [],[],[],[],[]
while not TimeStep.last():
  s.append(environment._physics._state)
  o.append(TimeStep.observation.tolist())
  r.append(TimeStep.reward)
  t.append(environment.physics.time())
  actions = agent.select_action(np.float32(TimeStep.observation)) 
  a.append(actions)
  TimeStep = environment.step(actions)   

We can now observe how the agent has learnt to control the $w_{in}$ to follow the target water level desired. We recall that the reward was designed as a Normal distribution centered on the target, and with $\sigma = 0.05 m$. The trained policy achieved a tracking precision of $\sim  0.05 m$ as expected.

In [ ]:
# Plot observation
plt.plot(t,o)
plt.ylabel('observations')
plt.xlabel('time [s]')
plt.legend(['target','state'])


The instantaneous rewards during the simulation with the trained agent are always close to the maximum value $=1$, except during the step instant. In this moment, the time decay of the water level is limited by the physical time scales of the system.

In [ ]:
# Plot reward
plt.plot(t,r)
plt.ylabel('reward')
plt.xlabel('time [s]')

We can also investigate the actions produced by the policy. Since we used the `CanonicalSpecWrapper` to clip the actions by physical bounds specified in task, the trained policy will provide actions in the $[-1,1]$ canonical interval. We need therefore to transform the actions to convert the inflow in SI units in the $[0,maxinflow]$ interval. This convertion is done internally when using `EnvironmentLoop` during training.

In [ ]:
# Revert action from canonical representation [-1,1] to SI 
f = lambda x: (np.clip(np.asarray(x),-1,1) + 1)*task._maxinflow/2 
a = [f(x) for x in a]

# Plot reward
plt.plot(t,a)
plt.ylabel('w_in')
plt.xlabel('time [s]')

# Summary

In this tutorial we showed how to implement a simple physical ode environment  following the [dm_control](https://github.com/deepmind/dm_control) requirements. Then, we trained an mpo agent with [acme](https://github.com/deepmind/acme) framework to perform continuous action space control.

Using deep reinforcement learning for this simple task and environment is obviously an overkill. However an extremely simple environment allows to play with deep reinforcement learning solutions at low computational costs. On top of that, making use of high quality frameworks such as [acme](https://github.com/deepmind/acme) allows potentially to easily scale up and generalize the approach.